In [3]:
%pwd

'e:\\works\\Main Project\\Fall Detection Using MLOps\\research'

In [5]:
import os
os.chdir("E:\works\Main Project\Fall Detection Using MLOps")

In [6]:

from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [7]:
import sys
import os
sys.path.append(os.path.abspath("src"))

In [8]:
from src.ChestCancerClassifier.constants import *
from src.ChestCancerClassifier.utils.common import read_yaml, create_directories

In [9]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [10]:

import os
import zipfile
import gdown
from ChestCancerClassifier import logger
from ChestCancerClassifier.utils.common import get_size

In [11]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self)-> str:
        '''
        Fetch data from the url
        '''

        try: 
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id,zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e
        
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [12]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-01-26 20:26:20,749: INFO: common: yaml file: E:\works\Main Project\Fall Detection Using MLOps\src\ChestCancerClassifier\config\config.yaml loaded successfully]
[2025-01-26 20:26:20,755: INFO: common: yaml file: E:\works\Main Project\Fall Detection Using MLOps\src\ChestCancerClassifier\params.yaml loaded successfully]
[2025-01-26 20:26:20,759: INFO: common: created directory at: artifacts]
[2025-01-26 20:26:20,762: INFO: common: created directory at: artifacts/data_ingestion]
[2025-01-26 20:26:20,766: INFO: 1522849741: Downloading data from https://drive.google.com/uc?id=1rf6azR-SiwxFGdee5N3SIuMB-Rz3kCph into file artifacts/data_ingestion/data.zip]


InvalidURL: URL has an invalid label.